In [7]:
import hashlib
%run SharedParams.ipynb


In [8]:
class MerkleTree:
    
    def __init__(self, data: list):
        self.tree_height = int(math.log(len(data), 2))
        assert self.tree_height == math.log(len(data), 2)
        
        self.data = data
        self.merkle = dict()
        
        for i in range(self.tree_height, 0, -1):
            for j in range(2**i):
                index = bin(j)[2:].rjust(i, "0")
                if i == self.tree_height:
                    self.merkle[index] = hashlib.sha256(str(data[j]).encode("utf-8")).digest()
                else:
                    self.merkle[index] = hashlib.sha256(self.merkle[index+"0"] + self.merkle[index+"1"]).digest()
        self.merkle[""] = hashlib.sha256(self.merkle["0"] + self.merkle["1"]).digest()
        self.root = self.merkle[""]
    
    def get_path(self, data_index):
        path = list()
        index = bin(data_index)[2:].rjust(self.tree_height, "0")
        while index != "":
            parent_index = index[:-1]
            neighbor_index = parent_index + "0" if index[-1] == "1" else parent_index + "1"
            path.append(self.merkle[neighbor_index].hex())
            index = parent_index
        return self.data[data_index], path
    
    @staticmethod
    def auth_path(data, index, path, root, tree_height):
        orig_index = index
        assert len(path) == tree_height
        curr_hash = hashlib.sha256(str(data).encode("utf-8")).digest()
        for h in path:
            bytes_h = bytes.fromhex(h)
            curr_hash = hashlib.sha256(curr_hash + bytes_h if index%2 == 0 else bytes_h + curr_hash).digest()
            index = int(index / 2)
            
        assert curr_hash == root

In [12]:
merkle_tree = MerkleTree([1,2,3,4])
data, path = merkle_tree.get_path(0)
MerkleTree.auth_path(data, 0, path, merkle_tree.root, merkle_tree.tree_height)

{'00': b'k\x86\xb2s\xff4\xfc\xe1\x9dk\x80N\xffZ?WG\xad\xa4\xea\xa2/\x1dI\xc0\x1eR\xdd\xb7\x87[K', '01': b'\xd4s^:&^\x16\xee\xe0?Yq\x8b\x9b]\x03\x01\x9c\x07\xd8\xb6\xc5\x1f\x90\xda:fn\xec\x13\xab5', '10': b'N\x07@\x85b\xbe\xdb\x8b`\xce\x05\xc1\xde\xcf\xe3\xad\x16\xb7"0\x96}\xe0\x1fd\x0b~G)\xb4\x9f\xce', '11': b'K"ww\xd4\xdd\x1f\xc6\x1co\x88OHd\x1d\x02\xb4\xd1!\xd3\xfd2\x8c\xb0\x8bU1\xfc\xac\xda\xbf\x8a', '0': b'B\x95\xf7.\xeb\x1e5\x07\xb8F\x1e$\x0e;\x8d\x18\xc1\xe7\xbd/\x11"\xb1\x1f\xc9\xec@\xa6X\x94\x03\x1a', '1': b' \xabt}E\xa7y8\xa5\xb8L)D\xb8\xf55\\I\xf2\x1d\xb0\xc5IE\x1cb\x81\xc9\x1b\xa4\x8d\r', '': b'\xcdS\xa2\xceh\xe6Gl)Q.\xa5<9\\\x7f]\x8f\xbc\xb4aM\x89)\x8d\xb1N*[\xdbTV'}
1
['d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35', '20ab747d45a77938a5b84c2944b8f5355c49f21db0c549451c6281c91ba48d0d']


AssertionError: 